In [ ]:
!pip install tensorflow
!pip install datasets

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
dataset = load_dataset("cifar10")

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.preprocessing.image import img_to_array, load_img, smart_resize
import tensorflow as tf

model = tf.keras.applications.resnet50.ResNet50(
  include_top=False, # take any input shape, not just 224x224
  weights='imagenet', # use the imagenet pretrained model
  pooling='avg' # use global average pooling to get a 2048 vector from our image
)

def process_and_embed(dataset, model, batch_size = 512):
  num_samples = len(dataset)
  X = np.zeros((num_samples, 2048))
  y = np.zeros(num_samples)

  for start in range(0, num_samples, batch_size):
    end = min(start + batch_size, num_samples)
    batch_images = np.zeros((end - start, 224, 224, 3))
    for i in range(start, end):
      img = img_to_array(dataset[i]['img'])
      img = smart_resize(img, (224, 224))
      img = preprocess_input(img)
      batch_images[i - start] = img
      y[i] = dataset[i]['label']
    batch_processed = model.predict(batch_images)
    X[start:end] = batch_processed

  return X, y

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
X_train, y_train = process_and_embed(dataset['train'], model)
X_test, y_test = process_and_embed(dataset['test'], model)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
logistic_regression_model = LogisticRegression(max_iter = 100, multi_class='multinomial', solver='lbfgs', random_state=42)
logistic_regression_model.fit(X_train, y_train)
y_pred = logistic_regression_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9097
Confusion Matrix:
 [[923   4   9   4   3   1   2   5  39  10]
 [  7 950   0   2   0   0   1   3   7  30]
 [ 15   0 889  22  32   8  19   8   5   2]
 [  2   1  16 839  20  78  24  15   3   2]
 [  3   0  26  25 901   7  13  20   3   2]
 [  2   0   9  88  15 860   8  16   1   1]
 [  5   0  20  27   9   3 932   3   1   0]
 [  5   0   9  10  29  19   1 927   0   0]
 [ 33   5   3   1   0   0   2   0 949   7]
 [ 10  44   2   3   1   0   1   2  10 927]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      0.92      0.92      1000
         1.0       0.95      0.95      0.95      1000
         2.0       0.90      0.89      0.90      1000
         3.0       0.82      0.84      0.83      1000
         4.0       0.89      0.90      0.90      1000
         5.0       0.88      0.86      0.87      1000
         6.0       0.93      0.93      0.93      1000
         7.0       0.93      0.93      0.93      1000
         8.0       0.9

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n",classification_report(y_test, y_pred))

Accuracy: 0.8614
Confusion Matrix:
 [[889   7  14   7   5   2   2   6  53  15]
 [  7 930   0   3   1   0   0   1   3  55]
 [ 39   1 810  33  53  15  35   7   5   2]
 [  6   2  28 749  29 117  41  16   1  11]
 [  8   2  29  28 834   8  35  50   3   3]
 [  2   1  13 107  26 820   9  20   1   1]
 [  7   2  30  33  12   8 908   0   0   0]
 [ 13   4  16  25  41  33   3 856   7   2]
 [ 49  14   1   6   0   0   2   0 913  15]
 [ 21  60   0   4   1   0   0   0   9 905]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.89      0.87      1000
         1.0       0.91      0.93      0.92      1000
         2.0       0.86      0.81      0.83      1000
         3.0       0.75      0.75      0.75      1000
         4.0       0.83      0.83      0.83      1000
         5.0       0.82      0.82      0.82      1000
         6.0       0.88      0.91      0.89      1000
         7.0       0.90      0.86      0.88      1000
         8.0       0.9